In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
import time
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import hstack
import datetime as dt

import math
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 35, 40
%matplotlib inline

from hdbscan import HDBSCAN
import matplotlib.cm as cm
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import __version__
from plotly import graph_objs as go
from plotly.graph_objs import *

## Читаем данные

In [ ]:
df=pd.read_pickle('MG_Sales_customer.pickle',compression='gzip')

In [ ]:
#---------------------------
#выбираем покупателей для дальнейшего анализа
sales_sum=df.groupby('Покупатель')['Количество'].sum().sort_values()
#выкидываем со слишком большими продажами (сводные карты) и тех кто купил один раз
sales_sum.drop(sales_sum[(sales_sum>sales_sum.tail(6)[0])].index, inplace=True)#|(sales_sum==1)
customers_name=list(sales_sum.index)
del sales_sum

#делаем выборку
select=df.loc[(df['Покупатель'].isin(customers_name))&(df['Дата']>=(dt.datetime(2016,1,1))),['Покупатель','ПокупательПол','ПокупательДатаРождения','ВидИзделия','ПодвидИзделия','СтильДизайна','ВидДизайна','ОсновнойКамень','ГруппаТовара','Коллекция','ЦветМеталла','ТоварСреднийВес','Размер','Вес','Количество','Сумма','Дата','ПокупательОтказОтСМС']]
del customers_name
del df

#Подготовка датасета
#ЦветМеталла=list(map(lambda xx: xx,list(select['ЦветМеталла'].unique())))
def codeMetall(_str):    
    for str_split in _str.lower().split():
        if str_split=='серебро': return 0
        if str_split=='золото': return 10
        if str_split=='зол.': return 11
        if str_split=='платина': return 20
        if str_split=='сплав': return -10
    return -20


select['ПокупательПолКод']=select['ПокупательПол'].map(lambda xx: {'Ж':0, 'М':1, '<Неопределено>':None}[xx])
select['ЦветМеталлаКод']=select['ЦветМеталла'].map(lambda xx: codeMetall(xx))
select['ПокупательПолКод'].fillna(select['ПокупательПолКод'].median(),inplace=True)
select['ПокупательГодРождения']=select['ПокупательДатаРождения'].dt.year
select['ПокупательГодРождения']=select['ПокупательГодРождения'].map(lambda xx: None if xx<1917 else xx)
select['ПокупательГодРождения']=select['ПокупательГодРождения'].map(lambda xx: None if xx>2010 else xx)
#select['ПокупательГодРождения'].fillna(select['ПокупательГодРождения'].median(),inplace=True)
select.drop(['ПокупательДатаРождения','ЦветМеталла','ПокупательПолКод'],  axis=1, inplace=True)
#выборка колонок
numerical_columns = [c for c in select.columns if select[c].dtype.name != 'object']
categorial_columns = [c for c in select.columns if select[c].dtype.name == 'object']

numerical_columns=list(set(numerical_columns)-(set(['ПокупательГодРождения','Дата','Сумма'])))
categorial_columns=list(set(categorial_columns)-(set(['ПокупательПол','ПокупательОтказОтСМС'])))



#Dummy-кодирование и шкалируем
lb_style = LabelBinarizer(sparse_output=True)
concList=[]
for col in categorial_columns:
    concList.append(lb_style.fit_transform(select[col]))    
concList.append(StandardScaler().fit_transform(select[numerical_columns]))#добавляем шклированные значения числовых переменных
X=hstack(concList)

del concList
print('shape ',X.shape)
print('Prepare finished')

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=int(math.sqrt(X.shape[1])), n_iter=5)
svd_representation = svd.fit_transform(X)
var1=np.cumsum(np.round(svd.explained_variance_ratio_, decimals=5)*100)
#plt.plot(var1[-50:])

#расчитываем оптимальное количество компонент
#более 90% дисперсии и шаг приращения каждой следующей компоненты <10^-4
optimal_n=np.intersect1d(np.argwhere(var1>90.),np.argwhere(svd.explained_variance_ratio_<=10**-4))[0]
print(optimal_n)#171

if optimal_n==None:
    raise 'Not enough n_components!'

svd = TruncatedSVD(n_components=optimal_n, n_iter=7)
svd_representation = svd.fit_transform(X)
print('reduced')
del var1

In [ ]:
del X

In [ ]:
sel_size=svd_representation.shape[0]
#k=.05#n_clusters = 3
#k=.01#n_clusters = 30
#k=.05#n_clusters = 6
#k=.03#n_clusters = 6
#k=.009#n_clusters = 33
#k=.01#n_clusters = 33
#k=.02#n_clusters = 33
#k=.03#n_clusters = 6 
#k=.02#n_clusters = 8
k=.025#n_clusters = 6 

k=.025#n_clusters = 4
k=.01#n_clusters = 12
k=.02#n_clusters = 8

k= 0.02#n_clusters = 3
k= 0.01#n_clusters = 15
k= 0.015#n_clusters = 4
k= 0.012#n_clusters = 

print('k=',k)
hdb_t1 = time.time()
#hdb = MiniBatchKMeans(n_clusters=n_clusters,max_iter=150,max_no_improvement=15,batch_size=165,tol=.01,n_init=5,random_state=17).fit(svd_representation)
#hdb = MiniBatchKMeans(n_clusters=n_clusters,max_iter=150,max_no_improvement=15,batch_size=260,tol=.0094,n_init=5,random_state=17).fit(svd_representation)
hdb = HDBSCAN(min_cluster_size=int(sel_size*k),metric='manhattan',core_dist_n_jobs=8).fit(svd_representation)#,metric='manhattan' alpha=.00001,alpha=.1,min_samples=1


hdb_elapsed_time = time.time() - hdb_t1
print('HDBSCAN Elapsed time to cluster: %.1f m' % (hdb_elapsed_time/60))

n_clusters = len(set(hdb.labels_)) - (1 if -1 in hdb.labels_ else 0)
print("n_clusters =", n_clusters)

In [ ]:
from sklearn.model_selection import ShuffleSplit
#svd_representation=X.copy()

ss = ShuffleSplit(n_splits=1, train_size=int(sel_size*.3))
subs= ss.split(svd_representation)#,hdb.labels_


for index in subs:
        X=svd_representation[index[0]]#.tocsr()
    #for _min_cluster_size in range(100, 700, 50):
        #print('min_cluster_size=',_min_cluster_size)
        #hdb_t1 = time.time()
        #hdb = HDBSCAN(min_cluster_size=_min_cluster_size,core_dist_n_jobs=8,metric='manhattan').fit(X)#,min_samples=1,alpha=.0001

        #hdb_elapsed_time = time.time() - hdb_t1
        #print('HDBSCAN Elapsed time to cluster: %.1f m' % (hdb_elapsed_time/60))

        n_clusters = len(set(hdb.labels_)) - (1 if -1 in hdb.labels_ else 0)
        print("n_clusters =", n_clusters)

        cluster_labels=hdb.labels_[index[0]]

        hdb_t1 = time.time()

        fig, (ax1) = plt.subplots(1, 1)
        fig.set_size_inches(12, 8)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 
        ax1.set_xlim([-.5, .5])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, X.shape[0] + (n_clusters + 1) * 12])

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print('Elapsed time to cluster: %6.1f m' % ((time.time()-hdb_t1)/60))
        print("For n_clusters =", n_clusters,
                      "The average silhouette_score is :", silhouette_avg)
        
        if silhouette_avg<=0: continue

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)
        n_len=len(sample_silhouette_values[sample_silhouette_values<0])
        one_per=len(sample_silhouette_values)/100
        percent=round(n_len/one_per,2)
        print('Negative silhouette count: ',n_len,', ',percent,'%')
        #if percent>=2: raise Exception('Плохой результат силуэта при кластеризации')

        y_lower = 12
        for i in range(n_clusters):
                    # Aggregate the silhouette scores for samples belonging to
                    # cluster i, and sort them
                    ith_cluster_silhouette_values = \
                        sample_silhouette_values[cluster_labels == i]

                    ith_cluster_silhouette_values.sort()

                    size_cluster_i = ith_cluster_silhouette_values.shape[0]
                    y_upper = y_lower + size_cluster_i

                    color = cm.spectral(float(i) / n_clusters)
                    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                                      0, ith_cluster_silhouette_values,
                                      facecolor=color, edgecolor=color, alpha=0.7)

                    # Label the silhouette plots with their cluster numbers at the middle
                    ax1.text(-0.5, y_lower + 0.5 * size_cluster_i, str(i))

                    # Compute the new y_lower for next plot
                    y_lower = y_upper + 12  # 10 for the 0 samples

        set_xlabel="The avg silhouette coefficient values = "+str(round(silhouette_avg,3))
        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel(set_xlabel)
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks(list(np.arange(-.6,1,.2)))

        plt.suptitle(("Silhouette analysis for HDBSCAN clustering on sample data "
                              "with n_clusters = %d" % n_clusters),
                             fontsize=14, fontweight='bold')

        plt.show()
        
        #if n_clusters<=6: break

del X
del cluster_labels

In [ ]:
select['ЦветМеталлаКод']=select['ЦветМеталлаКод'].map(lambda xx: {0: 'серебро', 10: 'золото', 11: 'золото', 20: 'платина',-10: 'сплав', -20: 'прочее'}[xx])
#select['ПокупательПолКод']=select['ПокупательПолКод'].map(lambda xx: {0: 'Ж', 1: 'М'}[xx])

In [ ]:
comma=','

#Выводим данные по кластерам в сводную таблицу
clusters_data=pd.DataFrame(columns=select.columns)
    
for cl in range(n_clusters):
    tsel=tsel=select.loc[hdb.labels_==cl]#выборка по кластеру
    sel=tsel.describe(include='all')
    clust_info=sel.loc[['top','mean']]
    
    
    cat_lbls=list(clust_info.loc['top',pd.notnull(clust_info.loc['top'])].index)#категориальные переменные
    #Рассчитываем три максимальных значения
    for col in cat_lbls:
        if col=='ПокупательПол': continue
        
        if col=='ПодвидИзделия':
            str_vals=[]            
            for val in clust_info.loc['top','ВидИзделия'].split(','):
                str_vals.append(tsel.loc[tsel['ВидИзделия']==val].groupby(col)['Покупатель'].count().sort_values(ascending=False).head(1).index.values[0])
        else:
            str_vals=list(tsel.groupby(col)['Покупатель'].count().sort_values(ascending=False).head(3).index)
            
        #clust_info.loc['top',col]=str_vals.translate(trantab)
        try:
            clust_info.loc['top',col]=comma.join(str_vals)
        except:
            clust_info.loc['top',col]=''
            
    
    clusters_data.loc[cl+1]=pd.concat([clust_info.fillna('').sum(axis=0),clust_info.fillna(0).sum(axis=0)])
    
    sex_count=tsel.groupby('ПокупательПол')['Покупатель'].nunique()
    perc1=sex_count.sum()/100
    clusters_data.loc[cl+1,'М (%)']=round(sex_count['М']/perc1,2)
    clusters_data.loc[cl+1,'Ж (%)']=round(sex_count['Ж']/perc1,2)
    
    sms_count=tsel.groupby('ПокупательОтказОтСМС')['Покупатель'].nunique()
    perc1=sms_count.sum()/100
    clusters_data.loc[cl+1,'Согласные с рассылкой СМС всего (%)']=round(sms_count[False]/perc1,2)
    
    sales_count=tsel.groupby('Покупатель')['Дата'].nunique()#Покупатели по датам    
    cust_names=sales_count[sales_count>1].index.values#Имена покупателей совершивших более одной покупки
    sms_count=tsel.loc[(tsel['Покупатель'].isin(cust_names))].groupby('ПокупательОтказОтСМС')['Покупатель'].nunique()
    perc1=sms_count.sum()/100
    clusters_data.loc[cl+1,'Согласные с рассылкой СМС при повторных покупках (%)']=round(sms_count[False]/perc1,2)
    
    #clusters_data.loc[cl+1,'Количество']=sel.loc['count','Покупатель']
    #clusters_data.loc[cl+1,'РазбросСумма']=sel.loc['std','Сумма']
    
    clusters_data.loc[cl+1,'Средний возраст (лет)']=dt.datetime.now().year-int(clusters_data.loc[cl+1,'ПокупательГодРождения'])
    
    clusters_data.loc[cl+1,'Уникальных покупателей в кластере']=sel.loc['unique','Покупатель']
    sales_count=tsel.groupby('Покупатель')['Количество'].sum()
    clusters_data.loc[cl+1,'Кол-во купленных изделий среднее']=sales_count.mean()
    
    sales_count=tsel.groupby('Покупатель')['Дата'].nunique()#Покупатели по датам
    clusters_data.loc[cl+1,'Число покупок среднее']=sales_count.mean()
    clusters_data.loc[cl+1,'2 Покупки']=sales_count[sales_count==2].shape[0]
    clusters_data.loc[cl+1,'3 Покупки']=sales_count[sales_count==3].shape[0]
    clusters_data.loc[cl+1,'4 Покупки']=sales_count[sales_count==4].shape[0]
    clusters_data.loc[cl+1,'5 Покупок']=sales_count[sales_count==5].shape[0]
    clusters_data.loc[cl+1,'>5 Покупок']=sales_count[sales_count>5].shape[0]
    
    
    sales_count_sum=tsel.groupby('Покупатель')['Сумма'].sum()
    clusters_data.loc[cl+1,'Средний чек']=(sales_count_sum/sales_count).mean()
    clusters_data.loc[cl+1,'Цена изделия средняя']=sel.loc['50%','Сумма'] 
    
    clusters_data.loc[cl+1,'Размер среднее']=clusters_data.loc[cl+1,'Размер']
    clusters_data.loc[cl+1,'Вес среднее']=clusters_data.loc[cl+1,'Вес']
    
    
clusters_data.drop(['ПокупательГодРождения','Количество','ПокупательОтказОтСМС','ПокупательПол','Сумма','ТоварСреднийВес','Размер','Вес','Дата'],axis=1,inplace=True)
#clusters_data.sort_values('Уникальных покупателей в кластере',inplace=True, ascending=False)

#clusters_data=clusters_data.reset_index()
#clusters_data.drop('index', axis=1, inplace=True)
clusters_data.index = range(1,clusters_data.shape[0]+1)

for col in cat_lbls:
    if col=='Покупатель':
        clusters_data=clusters_data.rename(columns={col: col+' - основные в кластере'})
    else:
        clusters_data=clusters_data.rename(columns={col: col+' - 3 самых популярных'})
    
    
    
writer = pd.ExcelWriter('Кластера по продажам.xlsx')
clusters_data.to_excel(writer,'Sheet1',index_label='№ кластера')
writer.save()

clusters_data

select.loc[hdb_labels==0].groupby('ВидИзделия')['Количество'].sum().sort_values(ascending=False)

In [ ]:
sms_count=select.groupby('ПокупательОтказОтСМС')['Покупатель'].nunique()

In [ ]:
sms_count

In [ ]:
df.columns

In [ ]:
list(range(600, 4000, 100))

In [ ]:
clusters_data.iloc[:,:15]